In [3]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [4]:
import hail as hl

In [5]:
hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38') 

23/08/26 23:07:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0056:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230826-2307-0.2.113-cf32652c5077.log


In [4]:
#read in polish alleles
poles = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/resources/polish-genomes/polish-genomes.mt/')

In [5]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_only.mt')

In [6]:
mt.count()

(4628319, 185)

In [7]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

2023-08-24 12:59:25.402 Hail: INFO: Reading table to impute column types
2023-08-24 12:59:26.510 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)


In [8]:
test = mt.filter_cols(mt.phenotypes.family == '.')

In [9]:
test.aggregate_cols(hl.agg.counter(test.phenotypes.sex))

{'F': 6, 'M': 34}

In [10]:
poles = poles.rows()
poles = hl.MatrixTable.from_rows_table(poles)
poles = poles.annotate_cols(col_idx = hl.missing(hl.tint32))
poles = poles.key_cols_by(poles.col_idx)

In [11]:
polish_samples = []

for i in range(1,3001):
    polish_samples.append(str(i)+'_polish')

In [12]:
polish_columns = hl.utils.range_matrix_table(n_rows=0, n_cols=3000)
polish_columns = polish_columns.annotate_rows(locus = hl.locus(hl.missing(hl.tstr), hl.missing(hl.tint32), reference_genome='GRCh38'),
                                              alleles = hl.missing(hl.tarray(hl.tstr)))
polish_columns = polish_columns.key_rows_by(polish_columns.locus, polish_columns.alleles)

In [13]:
polish_columns.write(localfs_path+'poles-1.mt')

2023-08-24 12:59:30.706 Hail: INFO: wrote matrix table with 0 rows and 3000 columns in 0 partitions to /localfs/4579427/poles-1.mt


In [14]:
poles = poles.union_cols(polish_columns, row_join_type='outer')

In [ ]:
poles.write(localfs_path+'poles-2.mt')

In [ ]:
poles = hl.read_matrix_table(localfs_path+'poles-2.mt')

In [ ]:
poles = poles.filter_rows(hl.is_defined(mt.rows()[poles.row_key])) #change this! create the matrix table with rows from the MT and draw based on poles. If not defined - assign ZERO (ask Marcin)

In [ ]:
poles = poles.checkpoint(localfs_path+'poles-3.mt')

In [ ]:
poles = poles.annotate_entries(AD = hl.missing(hl.tarray(hl.tint32)),
                         DP = hl.missing(hl.tint32),
                         GQ = hl.missing(hl.tint32),
                         GT = hl.call(hl.int32(hl.rand_bool(poles.info['AF'][0])), hl.int32(hl.rand_bool(poles.info['AF'][0]))),
                         MIN_DP = hl.missing(hl.tint32),
                         PGT = hl.call(hl.int32(hl.rand_bool(poles.info['AF'][0])), hl.int32(hl.rand_bool(poles.info['AF'][0]))),
                         PID = hl.missing(hl.tstr),
                         PL = hl.missing(hl.tarray(hl.tint32)),
                         PS = hl.missing(hl.tint32),
                         RGQ = hl.missing(hl.tint32),
                         SB = hl.missing(hl.tarray(hl.tint32)))

In [ ]:
poles = poles.checkpoint(localfs_path+'poles-4.mt')

In [ ]:
poles = poles.annotate_cols(
    s = hl.literal(polish_samples)[poles.col_idx]
)

In [ ]:
poles = poles.checkpoint(localfs_path+'poles-6.mt')

In [ ]:
poles = poles.key_cols_by(poles.s)

In [ ]:
poles = poles.checkpoint(localfs_path+'poles-7.mt')

In [ ]:
poles.count()

In [ ]:
poles = poles.naive_coalesce(500)

In [ ]:
#poles.write(
#    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic-controls-no-zeros.mt',
#    overwrite = True)

poles.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic-controls-no-zeros-3k.mt',
    overwrite = True)

### Make the simulated controls gender equal with the samples

In [4]:
poles = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic-controls-no-zeros.mt')
pl_mts = [poles]

PAR coordinates in GrCh38: http://m.ensembl.org/info/genome/genebuild/human_PARS.html

In [5]:
poles.count()

(3042073, 1200)

In [6]:
par1 = [10001,2781479]
par2 = [155701383,156030895]

nonpars = [[1,10000],[2781480,155701382]]

polish_samples = []

for i in range(1,3001):
    polish_samples.append(str(i)+'_polish')

poles_females = polish_samples[0:170] 
poles_males = polish_samples[170:]

path_names = ['poles']

for idx, mt in enumerate(pl_mts):
    mt = mt.annotate_cols(
        sex = hl.if_else(
            hl.array(poles_females).contains(mt.s),
            'F',
            'M'
        )
    )
    
    mt = mt.annotate_entries(
        GT = hl.if_else(
            mt.locus.contig == 'chrX',
            hl.if_else(
                mt['sex'] == 'F',
                mt.GT,
                hl.if_else(
                    mt.locus.position > nonpars[1][1],
                    mt.GT,
                    hl.if_else(
                        mt.locus.position < nonpars[0][1],
                        hl.if_else(
                            mt.locus.position > nonpars[0][0],
                            mt.GT,
                            hl.call(mt.GT[0], mt.GT[0])),
                        hl.call(mt.GT[0], mt.GT[0])))),
            mt.GT
        )
    )
    
    mt = mt.checkpoint(localfs_path+path_names[idx])  

2023-08-26 22:58:13.633 Hail: INFO: wrote matrix table with 3042073 rows and 1200 columns in 2 partitions to /localfs/4595503/poles


In [22]:
pl = hl.read_matrix_table(xxx)

In [23]:
pl = pl.naive_coalesce(500)

In [ ]:
pl.write(xxx)

### Join with existing samples

In [ ]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_only.mt')

mt = mt.drop(mt.group, mt.sample_qc)

mt = mt.select_entries(
    AD = mt.LAD,
    DP = mt.DP,
    GQ = mt.GQ,
    GT = mt.GT,
    MIN_DP = mt.MIN_DP,
    PGT = mt.LPGT,
    PID = mt.PID,
    PL = mt.LPL,
    PS = mt.PS,
    RGQ = mt.RGQ,
    SB = mt.SB
)

mt = mt.annotate_cols(sex = hl.missing(hl.tstr))
mt = mt.naive_coalesce(500)

#mt = mt.checkpoint('/localfs/4579427/mt_for_union.mt')
mt = hl.read_matrix_table('/localfs/4579427/mt_for_union.mt')
pl = hl.read_matrix_table('/localfs/4579427/pl_for_union.mt')

pl = pl.drop(pl.col_idx)
pl = mt.union_cols(pl)
pl = pl.checkpoint(localfs_path+'polesunion4.mt')

In [11]:
pl.aggregate_cols(hl.agg.counter(pl.sex))

{'F': 450, 'M': 2550, None: 185}